# 1. Framing the Portfolio Selection Problem

In [1]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np

In [2]:
########################################
########### IMPORT DATA ################
########################################

# Import drug data using pandas
data = pd.read_csv('drugs.csv', index_col=0)
projects = data.columns
t_area = data.iloc[0]
ttm = data.iloc[1]
ret = data.iloc[2]
cost = data.iloc[3]

#import covariance matrix
cov=pd.read_csv('drugs_cov.csv', index_col=0)

#therapeutic areas
ther=[
"Oncology", "Cardiovascular", "Respiratory and dermatology", "Transplantation",
"Rheumatology and hormone therapy", "Central nervous system", "Ophtalmics"]

#budget constraints for therapeutic areas
t_bud={"Oncology": 100,
       "Cardiovascular": 200,
       "Respiratory and dermatology": 150,
       "Transplantation": 100,
       "Rheumatology and hormone therapy": 300,
       "Central nervous system": 100,
       "Ophtalmics": 50}

interest_rate=0.03

In [4]:
########################################
########### MODEL ######################
########################################

# Create an empty model
m = gp.Model('portfolio')

# ADD DECISION VARIABLES HERE
vars = pd.Series(m.addVars(projects, vtype=GRB.BINARY, name="projects"))

########################################
#### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD CONSTRAINTS HERE

# Maximum Cost by Therapeutic Area
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == "Oncology") <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Cardiovascular') <= 200)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Respiratory and dermatology') <= 150)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Transplantation') <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Rheumatology and hormone therapy') <= 300)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Central nervous system') <= 100)
m.addConstr(sum(vars[i]*cost[i] for i in range(len(t_area)) if t_area[i] == 'Ophtalmics') <= 50)


# Pipeline Constraints
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["1"]) >= 0.15*sum(vars[t] for t in range(len(projects))))
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["2", "3"]) >= 0.2*sum(vars[t] for t in range(len(projects))))
m.addConstr(sum(vars[t] for t in range(len(projects)) if ttm[t] in ["4", "5"]) >= 0.25*sum(vars[t] for t in range(len(projects))))


# ADD OBJECTIVE FUNCTION HERE
obj = ret.T @ vars + (1000-(cost.T @ vars))*(1+interest_rate)
m.setObjective(obj, GRB.MAXIMIZE)

m.setParam('OutputFlag', 0)
m.optimize()

# ADD PRINTING HERE:
# Print optimal value of the objective function
print('\nValue of objective function \nExpected Return: \n$%g' % m.objVal,"(Million)")

# Print optimal values for the decision variables
print('\nDecision variables \nSelected Projects:\n')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Print the number of funded projects
temp=0
for v in m.getVars():
    if v.x>0.5:
        temp=temp+1
print('\nNumber of selected projects:', temp)

# Print total cost (buget spent on projects)
total_cost = round((vars.T @ cost).getValue(), 4)
print('\nTotal cost: $%g' % total_cost , "(Million)")

# Print the proportion of the budget spent on projects
print('\nPercent of the overall budget used for drug development:\n', round(total_cost/10, 4), "%")

# Print the portfolio variance when maximizing return  
portfolio_stdDev = sqrt((np.dot(np.dot(vars.T, cov), vars)).getValue())
print("\nPortfolio standard deviation when maximizing return:", round(portfolio_stdDev,4))


Value of objective function 
Expected Return: 
$23142.1 (Million)

Decision variables 
Selected Projects:

projects[1] = 0
projects[2] = 0
projects[3] = 1
projects[4] = 1
projects[5] = 0
projects[6] = 1
projects[7] = 0
projects[8] = 0
projects[9] = 0
projects[10] = 0
projects[11] = 0
projects[12] = 0
projects[13] = 1
projects[14] = 0
projects[15] = 1
projects[16] = 0
projects[17] = 1
projects[18] = 1
projects[19] = 0
projects[20] = 1
projects[21] = 1
projects[22] = 1
projects[23] = 0
projects[24] = 1
projects[25] = 1
projects[26] = 0
projects[27] = 1
projects[28] = 1
projects[29] = 1
projects[30] = 1
projects[31] = 0
projects[32] = 0
projects[33] = 0
projects[34] = 0
projects[35] = 0
projects[36] = 0
projects[37] = 0
projects[38] = 0
projects[39] = 1
projects[40] = 1
projects[41] = 0
projects[42] = 1
projects[43] = 1
projects[44] = 1
projects[45] = 0
projects[46] = 0
projects[47] = 1
projects[48] = 1
projects[49] = 0
projects[50] = 1
projects[51] = 0
projects[52] = 0
projects[53] = 0
